In [1]:
import requests
import pandas as pd 
import json
import concurrent.futures
import re
import queue
import os
from bs4 import BeautifulSoup
import lxml

from collections import defaultdict
import unicodedata

In [2]:
#URL pre-precess

import sys
if sys.version_info.major == 3:
    from urllib.parse import urlencode, urlparse, urlunparse, parse_qs, urljoin
else:
    from urllib import urlencode
    from urlparse import urlparse, urlunparse, parse_qs, urljoin

In [2]:
__name = pd.read_csv("allocate/VĂN HỌC.csv")
__name.columns

Index(['tên sách', 'ảnh bìa', 'thể loại', 'tác giả', 'nội dung tóm tắt',
       'giá bìa', 'công ty phát hành', 'nhà xuất bản', 'ngày xuất bản',
       'kích thước', 'loại bìa', 'số trang', 'dịch giả', 'path', 'phiên bản',
       'thương hiệu', 'xuất xứ', 'xuất xứ thương hiệu'],
      dtype='object')

In [3]:
df2 = pd.read_csv("DATA_v2.csv")
df2.dropna(inplace = True) 
df2.columns = ["Thể loại", "Nguồn nhập", "type"]

# Remove all row that not have fahasa
df2 = df2[df2['Nguồn nhập'].str.contains("fahasa")]
group_f1 = df2.groupby("Thể loại")["Nguồn nhập"].apply(list).to_dict()
group_f2 = df2.groupby("type")["Thể loại"].apply(list).to_dict()

n_dict = defaultdict(dict)
for k, v in group_f2.items():
    for stype in v:
        if type(group_f1[stype]) == list:
            _url = group_f1[stype][0]
        else:
            _url = group_f1[stype]
            
        if "order=num_orders" not in _url:
            print(group_f1[stype])
            group_f1[stype] = _url + '?order=num_orders&limit={}&p={}'
        else:
            group_f1[stype] = _url.split('?')[0] + '?order=num_orders&limit={}&p={}'
        n_dict[k][stype] = group_f1[stype]

['https://www.fahasa.com/foreigncategory.html']


In [30]:
n_dict

defaultdict(dict,
            {'CHĂM SÓC GIA ĐÌNH': {'Cẩm nang làm cha mẹ': 'https://www.fahasa.com/sach-trong-nuoc/nuoi-day-con/cam-nang-lam-cha-me.html?order=num_orders&limit={}&p={}',
              'Phương pháp giáo dục trẻ các nước': 'https://www.fahasa.com/sach-trong-nuoc/nuoi-day-con/phuong-phap-giao-duc-cac-nuoc.html?order=num_orders&limit={}&p={}',
              'Phát triển kỹ năng – trí tuệ cho trẻ': 'https://www.fahasa.com/sach-trong-nuoc/nuoi-day-con/phat-trien-ky-nang-tri-tue-cho-tre.html?order=num_orders&limit={}&p={}',
              'Giáo dục trẻ tuổi dậy thì': 'https://www.fahasa.com/sach-trong-nuoc/nuoi-day-con/giao-duc-tre-tuoi-teen.html?order=num_orders&limit={}&p={}',
              'Sức khỏe dinh dưỡng cho trẻ': 'https://www.fahasa.com/sach-trong-nuoc/nuoi-day-con/dinh-duong-suc-khoe-cho-tre.html?order=num_orders&limit={}&p={}',
              'Phụ nữ mang thai': 'https://www.fahasa.com/sach-trong-nuoc/nuoi-day-con/danh-cho-me-bau.html?order=num_orders&limit={}&p={}'}

In [4]:
_category_path = "https://www.fahasa.com/sach-trong-nuoc/nuoi-day-con/cam-nang-lam-cha-me.html?order=num_orders&limit=48&p=1"
_product = "https://www.fahasa.com/chao-con-em-be-so-sinh-nuoi-con-khong-phai-la-cuoc-chien-2-quyen-1-tai-ban-2019.html"

In [5]:
url = "https://www.vinabook.com/"

payload={}
headers = {
  'authority': 'www.fahasa.com',
  'pragma': 'no-cache',
  'cache-control': 'no-cache',
  'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-user': '?1',
  'sec-fetch-dest': 'document',
  'accept-language': 'vi,en-US;q=0.9,en;q=0.8,vi-VN;q=0.7',
  'cookie': 'BPC2=2c54362845c35e2a0dd27414d8c0a746; BPC2Referrer='
}


response = requests.request("GET", _product, headers=headers, data=payload)

In [8]:
soup = BeautifulSoup(response.text, "html.parser")

In [9]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html lang="vi" xml:lang="vi" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Sách Chào Con, Em Bé Sơ Sinh - Nuôi Con Không Phải Là Cuộc Chiến 2 (Quyển 1) - Tái Bản 2019 - FAHASA.COM</title>
<meta content="Chào Con, Em Bé Sơ Sinh - Nuôi Con Không Phải Là Cuộc Chiến 2 (Quyển 1) - Tái Bản 2019, giảm giá 20%, Chào Con, Em Bé Sơ ..." name="description"/>
<meta content="fahasa.com, fahasa, sách, nhà sách, bookstore, nhà sách trên mạng, mua sách online, văn phòng phẩm, đồ chơi" name="keywords">
<meta content="INDEX,FOLLOW" name="robots"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="cbc2e84e0738473689487ead377f46f2" name="p:domain_verify">
<meta content="331898836987489" property=

In [16]:
_pages = soup.find('div',{'class':'pages'}).findAll('li')
_pages

AttributeError: 'NoneType' object has no attribute 'findAll'

In [77]:
re.findall(r'\d+', _pages[-2].text)[0]

'9'

In [33]:
s = re.sub("(?m)^\s+","", soup.h1.text.rstrip())

'Chào Con, Em Bé Sơ Sinh - Nuôi Con Không Phải Là Cuộc Chiến 2 (Quyển 1) - Tái Bản 2019'

In [40]:
soup.find("div",{"class":"product-view-image-product"}).img['src']

'https://cdn0.fahasa.com/media/catalog/product/cache/1/small_image/600x600/9df78eab33525d08d6e5fb8d27136e95/8/9/8935280902381.jpg'

In [53]:
unicodedata.normalize("NFKD",soup.findAll("span",{"class":"price"})[-1].text)

'99.000 đ'

In [13]:
_pages = soup.find('div',{'class':'pages'}).findAll('li')
re.findall(r'\d+', _pages[-2].text)[0]

IndexError: list index out of range

In [10]:
_books = soup.find('div',{'class':'product_view_tab_content_ad'})
table = _books.find('table')

In [11]:
df = pd.read_html(str(table))[0]
df

,0,1
0,Mã hàng,8935280902381
1,Tên Nhà Cung Cấp,Thái Hà
2,Tác giả,"Hachun Lyonnet, Hương Đỗ"
3,NXB,NXB Lao Động
4,Năm XB,2019
5,Trọng lượng (gr),200
6,Kích Thước Bao Bì,15.5 x 24
7,Số trang,198
8,Hình thức,Bìa Mềm
9,Sản phẩm hiển thị trong,Bộ: Nuôi Con Không Phải Là Cuộc Chiến


In [23]:
df.index[df[0].str.contains("Tác giả")].to_list()[0]

2

In [29]:
_big_dict = {}
booksQueue = queue.Queue()

class Crawler():
    def __init__(self, url, cname):
        self.url = url
        self.category_name = cname
        
    def crawl(self):
        _concurrent = self._bookshelf()
        
        print(_concurrent)
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()*3) as executor:
            executor.map(self._books, _concurrent)
            
    def _books(self, _url,):
        
        book = {}
        __res =  requests.request("GET",_url, headers=headers)
        _soup = BeautifulSoup(__res.text, "html.parser")
        _books = _soup.find('div',{'class':'product_view_tab_content_ad'})
        table = _books.find('table')
        
        df = pd.read_html(str(table))[0]
        
        book['tên sách'] = re.sub("(?m)^\s+","", _soup.h1.text.rstrip())
        book['ảnh bìa'] = _soup.find("div",{"class":"product-view-image-product"}).img['src']
        book['thể loại'] = self.category_name
        book['tác giả'] = df[1][df.index[df[0].str.contains("Tác giả")].to_list()[0]]
        book['nhà xuất bản'] = df[1][df.index[df[0].str.contains("NXB")].to_list()[0]]
        book['ngày xuất bản'] = df[1][df.index[df[0].str.contains("Năm XB")].to_list()[0]]
        book['kích thước'] = df[1][df.index[df[0].str.contains("Kích Thước Bao Bì")].to_list()[0]]
        book['số trang'] = df[1][df.index[df[0].str.contains("Số trang")].to_list()[0]]
        book['loại bìa'] = df[1][df.index[df[0].str.contains("Hình thức")].to_list()[0]]
        
        book['nội dung tóm tắt'] = _soup.find("div",{"id":"desc_content"}).text
        book['giá bìa'] = unicodedata.normalize("NFKD",_soup.findAll("span",{"class":"price"})[-1].text)
        
        print(book)
        booksQueue.put(book)
        print(f"Number of products in my pocket {booksQueue.qsize()}")
        
    def _bookshelf(self):
        
        _bookshelf = []
        print(self.url.format(48, 1))
        _res = requests.request("GET", self.url.format(48, 1), headers=headers)
        soup = BeautifulSoup(_res.text, "html.parser")
        _pages = soup.find('div',{'class':'pages'}).findAll('li')
        
        if len(_pages) == 0:
            _total = 1
        else:
            _total = int(re.findall(r'\d+', _pages[-2].text)[0])
        
        for _ in range(_total):
            _ =+1
#             print(f"total page of {self.urlkey} : {_total}")
            _res = requests.request("GET", self.url.format(48, _), headers=headers)
            soup = BeautifulSoup(_res.text, "html.parser")
            for _ in soup.findAll("div",{"class":"product images-container"}):
                _bookshelf.append(_.a['href'])
            
        return _bookshelf

In [120]:
_pages = soup.find('div',{'class':'pages'}).findAll('li')
_pages[-2]

<li><a onclick="catalog_ajax.Page_change(9)">9</a></li>

In [30]:
total = 0
t= Crawler('https://www.fahasa.com/sach-trong-nuoc/sach-hoc-ngoai-ngu/ngoai-ngu-khac.html?order=num_orders&limit={}&p={}','CHĂM SÓC GIA ĐÌNH')
# print(t.crawl())
print(t._books('https://www.fahasa.com/black-jack-tap-3-tang-kem-bookmark-giay.html'))

{'tên sách': 'Black Jack - Tập 3 - Tặng Kèm Bookmark Giấy', 'ảnh bìa': 'https://cdn0.fahasa.com/media/catalog/product/cache/1/small_image/600x600/9df78eab33525d08d6e5fb8d27136e95/n/x/nxbtre_full_18072021_100719.jpg', 'thể loại': 'CHĂM SÓC GIA ĐÌNH', 'tác giả': 'Osamu Tezuka', 'nhà xuất bản': 'NXB Trẻ', 'ngày xuất bản': '2021', 'kích thước': '17.6 x 11.3 cm', 'số trang': '200', 'loại bìa': 'Bìa Mềm', 'nội dung tóm tắt': '\nBlack Jack\nCâu chuyện xoay quanh việc chữabệnh cứu người và những vấn đề y khoa, trong đó nhân vật chính là Black Jack, một bác sĩ tài giỏi nhưng “quái dị”. Điểm đặc biệt của anh chính là một vết sẹo dài trên khuôn mặt mà bất cứ ai trông thấy cũng phải khiếp sợ. Những người bệnh tìm đến anh đa số là những người, hoặc người thân họ, mắc những căn bệnh lạ hay bệnh hiểm nghèo mà không có vị bác sĩ nào có thể chữa được. Họ tìm đến anh với hy vọng anh dùng tài phẫu thuật kỳ diệu của mình để giữ lại mạng sống cho họ. Mỗi tập truyện là những câu chuyện xoay quanh việc chữa 

In [110]:
for _name in n_dict.keys():
    booksQueue = queue.Queue()
    for k,v in n_dict[_name].items():
        print(k)
        t = Crawler(v, k)
        t.crawl()

    total += booksQueue.qsize()
    df = pd.DataFrame(columns=__name.columns,index=[0])
    while not booksQueue.empty():
        df = df.append(booksQueue.get(),ignore_index=True)

    df.to_csv('fahasa/fahasa_'+ _name+'.csv')

Cẩm nang làm cha mẹ
https://www.fahasa.com/sach-trong-nuoc/nuoi-day-con/cam-nang-lam-cha-me.html?order=num_orders&limit=48&p=1
['https://www.fahasa.com/chao-con-em-be-so-sinh-nuoi-con-khong-phai-la-cuoc-chien-2-quyen-1-tai-ban-2019.html', 'https://www.fahasa.com/nep-sinh-hoat-cho-be-yeu-nuoi-con-khong-phai-la-cuoc-chien-2-quyen-2-tai-ban-2019.html', 'https://www.fahasa.com/be-tho-tu-ngu-cha-me-thu-thai-nuoi-con-khong-phai-la-cuoc-chien-2-quyen-3-tai-ban-2019.html', 'https://www.fahasa.com/combo-nuoi-con-khong-phai-la-cuoc-chien-2-bo-3-cuon-tai-ban-2019.html', 'https://www.fahasa.com/nuoi-con-khong-phai-la-cuoc-chien-272922.html', 'https://www.fahasa.com/noi-sao-cho-tre-nghe-loi.html', 'https://www.fahasa.com/cho-den-mau-giao-thi-da-muon.html', 'https://www.fahasa.com/vo-cung-tan-nhan-vo-cung-yeu-thuong-tai-ban-2017.html', 'https://www.fahasa.com/lam-cha-me-tinh-thuc.html', 'https://www.fahasa.com/thay-doi-vi-con-thuoc-dang-tang-cha-me-thoi-4-0.html', 'https://www.fahasa.com/cha-me-lam-

Number of products in my pocket 1
Number of products in my pocket 2
Number of products in my pocket 3
Number of products in my pocket 4
Number of products in my pocket 5
Number of products in my pocket 6
Number of products in my pocket 7
Number of products in my pocket 8
Number of products in my pocket 9
Number of products in my pocket 10
Number of products in my pocket 11
Number of products in my pocket 12
Number of products in my pocket 13
Number of products in my pocket 14
Number of products in my pocket 15
Number of products in my pocket 16
Number of products in my pocket 17
Number of products in my pocket 18
Number of products in my pocket 19
Number of products in my pocket 20
Number of products in my pocket 21
Number of products in my pocket 22
Number of products in my pocket 23
Number of products in my pocket 24
Number of products in my pocket 25
Number of products in my pocket 26
Number of products in my pocket 27
Number of products in my pocket 28
Number of products in my pock

Number of products in my pocket 232
Number of products in my pocket 233
Number of products in my pocket 234
Number of products in my pocket 235
Number of products in my pocket 236
Number of products in my pocket 237
Number of products in my pocket 238Number of products in my pocket 239

Number of products in my pocket 240
Number of products in my pocket 241
Number of products in my pocket 242
Number of products in my pocket 243
Number of products in my pocket 244
Number of products in my pocket 245
Number of products in my pocket 246
Number of products in my pocket 247
Number of products in my pocket 248
Number of products in my pocket 249
Number of products in my pocket 250
Number of products in my pocket 251
Number of products in my pocket 252
Number of products in my pocket 253
Number of products in my pocket 254
Number of products in my pocket 255
Number of products in my pocket 256
Number of products in my pocket 257
Number of products in my pocket 258
Number of products in my poc

Number of products in my pocket 433
Number of products in my pocket 434
Number of products in my pocket 435
Number of products in my pocket 436
Number of products in my pocket 437
Number of products in my pocket 438
Number of products in my pocket 439Number of products in my pocket 440

Number of products in my pocket 441
Number of products in my pocket 442
Number of products in my pocket 443
Number of products in my pocket 444
Number of products in my pocket 445
Number of products in my pocket 446Number of products in my pocket 447

Number of products in my pocket 448Number of products in my pocket 449

Number of products in my pocket 450
Number of products in my pocket 451
Number of products in my pocket 452
Number of products in my pocket 453
Number of products in my pocket 454
Number of products in my pocket 455
Number of products in my pocket 456
Number of products in my pocket 457
Number of products in my pocket 458
Number of products in my pocket 459
Number of products in my poc

Number of products in my pocket 625
Number of products in my pocket 626
Number of products in my pocket 627
Number of products in my pocket 628
Number of products in my pocket 629
Number of products in my pocket 630
Number of products in my pocket 631
Number of products in my pocket 632
Number of products in my pocket 633
Number of products in my pocket 634
Number of products in my pocket 635
Number of products in my pocket 636
Number of products in my pocket 637
Number of products in my pocket 638
Number of products in my pocket 639
Number of products in my pocket 640
Number of products in my pocket 641
Number of products in my pocket 642
Number of products in my pocket 643
Number of products in my pocket 644Number of products in my pocket 645

Number of products in my pocket 646
Number of products in my pocket 647
Number of products in my pocket 648
Number of products in my pocket 649
Number of products in my pocket 650
Number of products in my pocket 651
Number of products in my poc

...

Number of products in my pocket 1
Number of products in my pocket 2
Number of products in my pocket 3
Number of products in my pocket 4
Number of products in my pocket 5
Number of products in my pocket 6
Number of products in my pocket 7
Number of products in my pocket 8
Number of products in my pocket 9
Number of products in my pocket 10
Number of products in my pocket 11
Number of products in my pocket 12
Number of products in my pocket 13
Number of products in my pocket 14
Number of products in my pocket 15
Number of products in my pocket 16
Number of products in my pocket 17
Sân khấu - Điện ảnh 
https://www.fahasa.com/sach-trong-nuoc/tieu-su-hoi-ky/nghe-thua-giai-tri.html?order=num_orders&limit=48&p=1
['https://www.fahasa.com/cau-chuyen-nghe-thuat-the-story-of-art.html', 'https://www.fahasa.com/thu-tinh-gui-mot-nguoi.html', 'https://www.fahasa.com/song-voi-nghe-nail.html', 'https://www.fahasa.com/chau-nhuan-phat-dai-hiep-hong-kong.html', 'https://www.fahasa.com/blackpink-k-pop-s-no

Number of products in my pocket 31
Number of products in my pocket 32
Number of products in my pocket 33
Number of products in my pocket 34
Thời trang – may - thêu
https://www.fahasa.com/sach-trong-nuoc/am-nhac-my-thuat-thoi-trang/thoi-trang.html?order=num_orders&limit=48&p=1
['https://www.fahasa.com/hoc-ve-bang-hinh-co-ban-nghe-thuat-that-la-don-gian-tai-ban-2018.html', 'https://www.fahasa.com/tu-lam-dep.html', 'https://www.fahasa.com/ky-thuat-cat-may-co-ban-thiet-ke-mot-so-chi-tiet-tren-trang-phuc-nu.html', 'https://www.fahasa.com/ky-thuat-cat-may-thiet-ke-thoi-trang-nam.html', 'https://www.fahasa.com/688-cach-dan-moc-ao-len-nam.html', 'https://www.fahasa.com/ky-thuat-dan-theu-moc-ao-choang-va-khan-len.html', 'https://www.fahasa.com/ky-thuat-moc-len-soi-thoi-trang-mua-ha.html', 'https://www.fahasa.com/ky-thuat-moc-len-soi-thoi-trang-mua-xuan.html', 'https://www.fahasa.com/thuc-hanh-dan-moc-len-soi.html']
Number of products in my pocket 35
Number of products in my pocket 36
Thủ công –

Number of products in my pocket 1
Number of products in my pocket 2
Number of products in my pocket 3
Number of products in my pocket 4
Number of products in my pocket 5
Number of products in my pocket 6
Number of products in my pocket 7
Number of products in my pocket 8
Number of products in my pocket 9
Number of products in my pocket 10
Number of products in my pocket 11
Number of products in my pocket 12
Number of products in my pocket 13
Number of products in my pocket 14
Number of products in my pocket 15
Number of products in my pocket 16
Number of products in my pocket 17
Number of products in my pocket 18
Number of products in my pocket 19
Number of products in my pocket 20Number of products in my pocket 21

Number of products in my pocket 22
Number of products in my pocket 23
Number of products in my pocket 24
Number of products in my pocket 25
Number of products in my pocket 26
Number of products in my pocket 27
Number of products in my pocket 28
Number of products in my pock

Number of products in my pocket 233
Number of products in my pocket 234
Number of products in my pocket 235
Number of products in my pocket 236
Number of products in my pocket 237
Number of products in my pocket 238
Number of products in my pocket 239
Number of products in my pocket 240
Number of products in my pocket 241
Number of products in my pocket 242
Number of products in my pocket 243
Number of products in my pocket 244
Number of products in my pocket 245
Number of products in my pocket 246
Number of products in my pocket 247
Number of products in my pocket 248
Number of products in my pocket 249
Number of products in my pocket 250Number of products in my pocket 251

Number of products in my pocket 252
Number of products in my pocket 253
Number of products in my pocket 254
Number of products in my pocket 255
Number of products in my pocket 256
Number of products in my pocket 257
Number of products in my pocket 258
Number of products in my pocket 259
Number of products in my poc

MissingSchema: Invalid URL 'h?order=num_orders&limit=48&p=1': No schema supplied. Perhaps you meant http://h?order=num_orders&limit=48&p=1?

In [9]:
_df = pd.read_csv("fahasa/fahasa_CHĂM SÓC GIA ĐÌNH.csv")
_df.head()

,Unnamed: 0,tên sách,ảnh bìa,thể loại,tác giả,nội dung tóm tắt,giá bìa,công ty phát hành,nhà xuất bản,ngày xuất bản,kích thước,loại bìa,số trang,dịch giả,path,phiên bản,thương hiệu,xuất xứ,xuất xứ thương hiệu,tên dách
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,https://cdn0.fahasa.com/media/catalog/product/...,Cẩm nang làm cha mẹ,NXB Lao Động,"\nChào Con, Em Bé Sơ Sinh - Nuôi Con Không Phả...",99.000 đ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Chào Con, Em Bé Sơ Sinh - Nuôi Con Không Phải ..."
2,2,NaN,https://cdn0.fahasa.com/media/catalog/product/...,Cẩm nang làm cha mẹ,NXB Tổng Hợp TPHCM,"\nCha Voi - Dạy Con Nên Người Ở Thời Đại Số\n""...",150.000 đ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cha Voi: Dạy Con Nên Người Ở Thời Đại Số
3,3,NaN,https://cdn0.fahasa.com/media/catalog/product/...,Cẩm nang làm cha mẹ,NXB Lao Động,\nNếp Sinh Hoạt Cho Bé Yêu - Nuôi Con Không Ph...,110.000 đ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nếp Sinh Hoạt Cho Bé Yêu - Nuôi Con Không Phải...
4,4,NaN,https://cdn0.fahasa.com/media/catalog/product/...,Cẩm nang làm cha mẹ,NXB Lao Động,"\nBé Thơ Tự Ngủ, Cha Mẹ Thư Thái - Nuôi Con Kh...",110.000 đ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bé Thơ Tự Ngủ, Cha Mẹ Thư Thái - Nuôi Con Khôn..."


In [8]:
_df = pd.read_csv("fahasa/fahasa_CHĂM SÓC GIA ĐÌNH.csv")
_df.rename(columns={'tên dách': 'tên sách'})
_df.head()

,Unnamed: 0,tên sách,ảnh bìa,thể loại,tác giả,nội dung tóm tắt,giá bìa,công ty phát hành,nhà xuất bản,ngày xuất bản,kích thước,loại bìa,số trang,dịch giả,path,phiên bản,thương hiệu,xuất xứ,xuất xứ thương hiệu,tên dách
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,https://cdn0.fahasa.com/media/catalog/product/...,Cẩm nang làm cha mẹ,NXB Lao Động,"\nChào Con, Em Bé Sơ Sinh - Nuôi Con Không Phả...",99.000 đ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Chào Con, Em Bé Sơ Sinh - Nuôi Con Không Phải ..."
2,2,NaN,https://cdn0.fahasa.com/media/catalog/product/...,Cẩm nang làm cha mẹ,NXB Tổng Hợp TPHCM,"\nCha Voi - Dạy Con Nên Người Ở Thời Đại Số\n""...",150.000 đ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cha Voi: Dạy Con Nên Người Ở Thời Đại Số
3,3,NaN,https://cdn0.fahasa.com/media/catalog/product/...,Cẩm nang làm cha mẹ,NXB Lao Động,\nNếp Sinh Hoạt Cho Bé Yêu - Nuôi Con Không Ph...,110.000 đ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nếp Sinh Hoạt Cho Bé Yêu - Nuôi Con Không Phải...
4,4,NaN,https://cdn0.fahasa.com/media/catalog/product/...,Cẩm nang làm cha mẹ,NXB Lao Động,"\nBé Thơ Tự Ngủ, Cha Mẹ Thư Thái - Nuôi Con Kh...",110.000 đ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bé Thơ Tự Ngủ, Cha Mẹ Thư Thái - Nuôi Con Khôn..."


In [2]:
import glob 
for _ in glob.glob("fahasa/*"):
    _df = pd.read_csv(_)
    
    print(_)

fahasa\fahasa_CHĂM SÓC GIA ĐÌNH.csv
fahasa\fahasa_FOREIGN BOOKS .csv
fahasa\fahasa_HỌC NGÔN NGỮ.csv
fahasa\fahasa_NGÀNH KHOA HỌC KỸ THUẬT – CÔNG NGHỆ.csv
fahasa\fahasa_NGÀNH KHOA HỌC TỰ NHIÊN.csv
fahasa\fahasa_NGÀNH KHOA HỌC XÃ HỘI VÀ NHÂN VĂN.csv
fahasa\fahasa_NGÀNH NGHỆ THUẬT.csv
fahasa\fahasa_NGÀNH THỂ DỤC – THỂ THAO.csv
fahasa\fahasa_NGÀNH Y HỌC.csv
fahasa\fahasa_NHÂN VẬT - SỰ KIỆN .csv
fahasa\fahasa_PHONG THỦY - KINH DỊCH.csv
fahasa\fahasa_SÁCH GIÁO TRÌNH.csv
fahasa\fahasa_SÁCH KINH TẾ.csv
fahasa\fahasa_SÁCH KỸ NĂNG.csv
fahasa\fahasa_SÁCH THAM KHẢO.csv
fahasa\fahasa_THIẾU NHI .csv
fahasa\fahasa_TẠP CHÍ - CATALOGUE.csv
fahasa\fahasa_TỪ ĐIỂN.csv
fahasa\fahasa_VĂN HỌC.csv
fahasa\fahasa_ĐẢNG - ĐOÀN - HỘI.csv
